In [1]:
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM

Using TensorFlow backend.
/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [6]:
# import data
cur = 'EURUSD'
raw_data = pd.read_csv('http://api.fxhistoricaldata.com/indicators?instruments=%s&expression=open,high,low,close&item_count=50000&format=csv&timeframe=hour' % cur)
# change columns
raw_data.columns = ['Currency', 'Datetime', 'open', 'high', 'low', 'close']
raw_data = raw_data.iloc[::-1]

In [7]:
# isolate currencies and drop useless columns
Currencies = raw_data.Currency.unique()
DataFrameDict = {elem : pd.DataFrame for elem in Currencies}
for key in DataFrameDict.keys():
    DataFrameDict[key] = raw_data[:][raw_data.Currency == key].drop(['Currency', 'Datetime'], axis=1)

In [16]:
def group(values, hours):
    data_grouped = []
    for i in range(len(values)-hours):
        data_grouped.append(values[i:i+hours])
    return np.array(data_grouped)

def sma(values, periods):
    return sum(values)/periods

closes = DataFrameDict[cur].values.transpose()[3]
sma_values = [sma(x, 14) for x in group(closes, 14)]

In [86]:
# group by tot hours
hours = 10
data_grouped = np.c_[DataFrameDict[cur].values[14:], np.expand_dims(sma_values, axis=1)]
data_grouped = group(data_grouped, hours)

In [87]:
# labels
labels = []
for i, scale in enumerate(data_grouped):
    open_p = scale[-1][-1]  # get the open price 
    try:
        close_p = data_grouped[i+1][-1][-1]  # get close price
    except IndexError:
        pass
    #labels.append((close_p - open_p) / 2)
    if (close_p - open_p) >= 0:
        labels.append(1)
    elif (close_p - open_p) < 0:
        labels.append(0)

In [88]:
# scale
data = []
for dataframe in data_grouped:
    scaler = MinMaxScaler(feature_range=(-1,1))
    scaler.fit(dataframe)
    data.append(np.array(scaler.transform(dataframe)))

In [89]:
def split(l, part):
    return (np.array(l[:part]), np.array(l[part:]))

x_train, x_test = split(data, -10000)
y_train, y_test = split(labels, -10000)

In [90]:
# test_labels
for x in range(20):
    res = x_test[x+1][-1][-1] - x_test[x][-1][-1]
    print("res: %.3f - label: %d" % (res, y_test[x]))

res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.000 - label: 0
res: 0.121 - label: 1
res: 0.203 - label: 1
res: 0.354 - label: 1


In [121]:
# neural network
model = Sequential()
model.add(LSTM(4, input_shape=(hours,5)))
model.add(Dense(20))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# For a mean squared error regression problem
model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['accuracy'])

In [122]:
model.fit(x_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
39975/39975 [==============================] - 10s 255us/step - loss: 0.1375 - acc: 0.8137
Epoch 2/10
39975/39975 [==============================] - 8s 208us/step - loss: 0.0999 - acc: 0.8615
Epoch 3/10
39975/39975 [==============================] - 8s 212us/step - loss: 0.0903 - acc: 0.8742
Epoch 4/10
39975/39975 [==============================] - 8s 207us/step - loss: 0.0849 - acc: 0.8829
Epoch 5/10
39975/39975 [==============================] - 8s 208us/step - loss: 0.0812 - acc: 0.8899
Epoch 6/10
39975/39975 [==============================] - 9s 213us/step - loss: 0.0788 - acc: 0.8936
Epoch 7/10
39975/39975 [==============================] - 8s 206us/step - loss: 0.0774 - acc: 0.8964
Epoch 8/10
39975/39975 [==============================] - 9s 221us/step - loss: 0.0765 - acc: 0.8967
Epoch 9/10
39975/39975 [==============================] - 9s 215us/step - loss: 0.0757 - acc: 0.8972
Epoch 10/10
39975/39975 [==============================] - 8s 210us/step - loss: 0.0752 - 

In [123]:
model.evaluate(x_test, y_test, batch_size=32)

10000/10000 [==============================] - 1s 116us/step


[0.069931745640560983, 0.90490000000000004]

In [124]:
predicted = model.predict(x_test)
actual = y_test
check_list = []
for x in range(len(predicted)):
    if actual[x] >= 0.8 or actual[x] <= 0.2:
        if abs(predicted[x] - actual[x]) <= 0.2:
            check_list.append(1)
        elif abs(predicted[x] - actual[x]) > 0.2:
            check_list.append(-1)
print('percentage - %f' % np.mean(check_list))
print('ones - %d' % len([x for x in check_list if x == 1]))

percentage - 0.566600
ones - 7833


In [125]:
for x in range(len(y_test)):
    print("%d - %g" % (y_test[x], predicted[x]))

0 - 0.0111585
0 - 0.012931
0 - 0.00933926
0 - 0.0106662
0 - 0.0117196
0 - 0.0174319
0 - 0.013679
0 - 0.0153934
0 - 0.0145059
0 - 0.0142538
0 - 0.0176816
0 - 0.0268498
0 - 0.0197011
0 - 0.0409434
0 - 0.0533129
0 - 0.837418
0 - 0.629034
1 - 0.93033
1 - 0.882382
1 - 0.957685
1 - 0.969888
1 - 0.973027
1 - 0.98941
1 - 0.983904
1 - 0.985477
1 - 0.962692
1 - 0.98185
1 - 0.974949
1 - 0.988718
1 - 0.959435
1 - 0.480645
1 - 0.983518
1 - 0.983761
1 - 0.967444
1 - 0.915739
1 - 0.930897
1 - 0.963007
0 - 0.150538
0 - 0.147697
0 - 0.12138
0 - 0.0567747
0 - 0.160343
1 - 0.715404
1 - 0.963275
1 - 0.997592
1 - 0.998202
1 - 0.993345
0 - 0.87222
0 - 0.157753
0 - 0.174809
0 - 0.178742
0 - 0.0181891
0 - 0.085931
1 - 0.0575521
1 - 0.144955
0 - 0.490294
0 - 0.379346
0 - 0.0179141
0 - 0.00253922
0 - 0.0110628
0 - 0.0151166
0 - 0.0103087
0 - 0.159656
0 - 0.145975
1 - 0.905692
1 - 0.926355
1 - 0.980256
1 - 0.987561
1 - 0.991982
1 - 0.992667
1 - 0.990708
1 - 0.985292
1 - 0.970942
1 - 0.993041
1 - 0.988324
1 - 0.9

1 - 0.657814
1 - 0.967395
0 - 0.288884
0 - 0.715045
0 - 0.275954
1 - 0.979521
1 - 0.997132
1 - 0.994671
1 - 0.951034
1 - 0.92603
1 - 0.902745
0 - 0.869112
1 - 0.696301
0 - 0.115062
1 - 0.171206
0 - 0.182756
0 - 0.00933729
0 - 0.00299731
0 - 0.00394528
0 - 0.00471741
0 - 0.00291809
0 - 0.00771992
0 - 0.0164782
0 - 0.0170617
0 - 0.0241569
0 - 0.0442091
0 - 0.0169937
0 - 0.0122527
0 - 0.0299579
0 - 0.0852332
0 - 0.527821
0 - 0.110897
0 - 0.15556
0 - 0.166142
0 - 0.453648
0 - 0.415101
0 - 0.406253
0 - 0.182597
1 - 0.655895
1 - 0.893703
1 - 0.846717
1 - 0.994235
1 - 0.998562
1 - 0.997366
1 - 0.991036
1 - 0.981403
1 - 0.980767
1 - 0.99037
1 - 0.968604
1 - 0.95943
1 - 0.977471
1 - 0.991923
1 - 0.98274
1 - 0.994592
1 - 0.987642
1 - 0.99236
1 - 0.982028
1 - 0.975379
1 - 0.967569
1 - 0.984526
1 - 0.983588
1 - 0.965704
1 - 0.89651
1 - 0.887865
1 - 0.552156
1 - 0.441966
1 - 0.583516
0 - 0.592253
0 - 0.196399
0 - 0.130031
0 - 0.0449116
0 - 0.0183383
0 - 0.00607552
0 - 0.0084739
0 - 0.00948393
0 - 0

0 - 0.0408196
0 - 0.0378043
0 - 0.0243927
0 - 0.0135952
0 - 0.0438544
0 - 0.0474835
0 - 0.0603872
0 - 0.0499768
0 - 0.0987454
0 - 0.116679
0 - 0.066348
0 - 0.134505
1 - 0.152859
1 - 0.917996
0 - 0.797434
1 - 0.632515
1 - 0.906311
1 - 0.96154
1 - 0.993878
1 - 0.969479
1 - 0.991676
1 - 0.981314
1 - 0.985684
1 - 0.871173
1 - 0.969664
1 - 0.530205
0 - 0.196724
1 - 0.921849
1 - 0.928181
1 - 0.922558
1 - 0.822697
0 - 0.819516
1 - 0.875625
1 - 0.829699
0 - 0.968739
1 - 0.233566
1 - 0.941394
1 - 0.982679
1 - 0.99427
1 - 0.987525
1 - 0.996184
1 - 0.996965
1 - 0.948112
1 - 0.96584
1 - 0.543217
1 - 0.639725
0 - 0.159141
0 - 0.0362468
0 - 0.0632435
0 - 0.13105
0 - 0.0121937
0 - 0.0388402
0 - 0.0581217
0 - 0.00977642
0 - 0.004834
0 - 0.00848594
0 - 0.0106027
0 - 0.013199
0 - 0.015015
0 - 0.0179639
0 - 0.0464348
0 - 0.088205
0 - 0.294421
0 - 0.067247
0 - 0.0375611
0 - 0.0154577
0 - 0.0321388
0 - 0.0112658
0 - 0.0161802
0 - 0.0400377
0 - 0.0212035
0 - 0.0180067
0 - 0.0524959
0 - 0.0487477
0 - 0.09933

1 - 0.960365
1 - 0.975931
1 - 0.961076
1 - 0.879365
1 - 0.430872
1 - 0.605986
0 - 0.909034
0 - 0.656976
1 - 0.863938
0 - 0.577872
1 - 0.782533
1 - 0.759949
1 - 0.936045
0 - 0.786827
1 - 0.787238
1 - 0.998187
1 - 0.997687
1 - 0.993502
1 - 0.991434
1 - 0.99142
1 - 0.985759
1 - 0.977988
1 - 0.977376
1 - 0.966411
1 - 0.978095
1 - 0.891997
1 - 0.988782
1 - 0.437778
1 - 0.988862
1 - 0.937924
1 - 0.733328
0 - 0.112221
0 - 0.0166653
0 - 0.0324906
0 - 0.0460571
0 - 0.0171314
0 - 0.0180633
0 - 0.0309076
0 - 0.0250917
0 - 0.0398679
0 - 0.0458695
0 - 0.0258185
0 - 0.106181
0 - 0.0996318
0 - 0.577296
1 - 0.722329
1 - 0.944453
1 - 0.947737
1 - 0.965514
1 - 0.964468
1 - 0.992182
1 - 0.987071
1 - 0.970295
1 - 0.838681
1 - 0.975258
1 - 0.464574
1 - 0.861147
1 - 0.84735
1 - 0.901278
0 - 0.735282
0 - 0.0796809
0 - 0.00416663
0 - 0.00202484
0 - 0.0240423
0 - 0.0206069
0 - 0.0129449
0 - 0.0171985
0 - 0.0376768
1 - 0.681446
1 - 0.680749
1 - 0.771745
1 - 0.777463
1 - 0.748245
1 - 0.839939
1 - 0.975578
1 - 0.

1 - 0.42412
0 - 0.301316
0 - 0.197492
1 - 0.434964
0 - 0.495412
0 - 0.0237496
0 - 0.0338046
0 - 0.00631382
0 - 0.00295852
0 - 0.00721262
0 - 0.00951616
0 - 0.0315876
0 - 0.00980906
0 - 0.039238
0 - 0.0163326
0 - 0.0901074
1 - 0.934853
1 - 0.854135
1 - 0.735895
1 - 0.76377
1 - 0.928786
1 - 0.962864
1 - 0.955736
1 - 0.980735
1 - 0.88819
1 - 0.926034
1 - 0.899758
1 - 0.989019
1 - 0.977931
1 - 0.982704
0 - 0.966196
0 - 0.660143
1 - 0.607409
1 - 0.869524
0 - 0.967809
1 - 0.609199
1 - 0.968649
1 - 0.97552
1 - 0.995788
1 - 0.99748
1 - 0.518336
0 - 0.860909
1 - 0.303201
0 - 0.171068
0 - 0.0458807
1 - 0.917453
1 - 0.997949
1 - 0.996525
1 - 0.994137
1 - 0.989457
1 - 0.986582
1 - 0.668241
0 - 0.666175
0 - 0.490249
1 - 0.431736
1 - 0.264304
1 - 0.414423
1 - 0.64656
1 - 0.897253
0 - 0.96208
0 - 0.00115826
0 - 0.00343345
0 - 0.00483226
0 - 0.00549308
0 - 0.00457575
0 - 0.00531196
0 - 0.0130432
0 - 0.0209735
0 - 0.0154631
0 - 0.0140831
0 - 0.0149669
0 - 0.014716
0 - 0.0191582
0 - 0.0146016
0 - 0.0150

0 - 0.489133
0 - 0.767138
0 - 0.134764
0 - 0.0339954
1 - 0.200392
0 - 0.0433242
0 - 0.0134999
0 - 0.0124222
0 - 0.00727535
0 - 0.00706139
0 - 0.0117895
0 - 0.0077903
0 - 0.00796582
0 - 0.0145377
0 - 0.0305732
0 - 0.346739
0 - 0.0444047
0 - 0.0650284
0 - 0.122113
0 - 0.0863556
0 - 0.194366
0 - 0.152098
0 - 0.198753
1 - 0.313565
0 - 0.3113
1 - 0.410278
1 - 0.147954
1 - 0.389014
0 - 0.221844
0 - 0.0101156
0 - 0.00106157
0 - 0.00397125
0 - 0.0136104
0 - 0.013575
0 - 0.00776687
0 - 0.0175909
0 - 0.0291171
0 - 0.154127
0 - 0.177589
0 - 0.259481
0 - 0.155876
0 - 0.124856
0 - 0.126189
0 - 0.287427
1 - 0.267958
1 - 0.704305
1 - 0.567938
0 - 0.216748
1 - 0.209767
1 - 0.821354
0 - 0.857556
0 - 0.415414
0 - 0.00195122
0 - 0.108087
1 - 0.564179
1 - 0.975781
1 - 0.997249
1 - 0.997738
1 - 0.9913
1 - 0.993184
1 - 0.9894
1 - 0.981381
1 - 0.989357
1 - 0.987331
1 - 0.986478
1 - 0.955679
1 - 0.976631
1 - 0.938156
1 - 0.945003
1 - 0.837541
1 - 0.894186
1 - 0.46051
1 - 0.184237
0 - 0.930547
0 - 0.0876807
0 

1 - 0.967136
1 - 0.977287
1 - 0.921757
1 - 0.981147
1 - 0.899089
1 - 0.929672
1 - 0.950981
1 - 0.920515
1 - 0.940862
1 - 0.920228
1 - 0.867086
1 - 0.731724
0 - 0.652945
0 - 0.569228
0 - 0.0416766
0 - 0.0490182
0 - 0.0178096
0 - 0.0171947
0 - 0.00452841
0 - 0.0307559
0 - 0.0250266
0 - 0.0771204
0 - 0.0112727
0 - 0.00886659
0 - 0.0639616
0 - 0.0471758
0 - 0.0711312
0 - 0.11857
0 - 0.0432466
0 - 0.0257459
0 - 0.044655
0 - 0.113392
0 - 0.390704
0 - 0.286375
0 - 0.0617531
0 - 0.026822
1 - 0.0099791
0 - 0.0372645
0 - 0.0139023
0 - 0.00633786
0 - 0.00431752
0 - 0.0101476
0 - 0.0329551
0 - 0.00960995
0 - 0.011713
0 - 0.0119588
0 - 0.0119078
0 - 0.0116968
1 - 0.0385254


In [126]:
model.evaluate(x_test[-2:-1], y_test[-2:-1])

1/1 [==============================] - 0s 3ms/step


[0.0001368160592392087, 1.0]

In [127]:
n = 20
predict = model.predict(x_test[-n:])
for x in range(-n,0):
    print("%.2f - %.2f" % (predict[x], y_test[x]))

0.03 - 0.00
0.04 - 0.00
0.11 - 0.00
0.39 - 0.00
0.29 - 0.00
0.06 - 0.00
0.03 - 0.00
0.01 - 1.00
0.04 - 0.00
0.01 - 0.00
0.01 - 0.00
0.00 - 0.00
0.01 - 0.00
0.03 - 0.00
0.01 - 0.00
0.01 - 0.00
0.01 - 0.00
0.01 - 0.00
0.01 - 0.00
0.04 - 1.00


In [144]:
test = []
test.append([1.20744, 1.20748, 1.20704, 1.20741])
test.append([1.20739, 1.20755, 1.20698, 1.20716])
test.append([1.20714, 1.20744, 1.20567, 1.20569])
test.append([1.20573, 1.20673, 1.20556, 1.20575])
test.append([1.20577, 1.20529, 1.20626, 1.20515])
test.append([1.20527, 1.20577, 1.20419, 1.20457])
test.append([1.20454, 1.20541, 1.20428, 1.20501])
test.append([1.20495, 1.20552, 1.20484, 1.20529])
test.append([1.20527, 1.20541, 1.20447, 1.20474])
test.append([1.20476, 1.20831, 1.20448, 1.20521])


In [145]:
def _scale(arr):
    # scale
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler.fit(arr)
    return np.array(scaler.transform(arr))

smas = [1.20716, 1.20710, 1.20707, 1.20703, 1.20691, 1.20682, 1.20664, 1.20651, 1.20640, 1.20622]
test = np.c_[test, np.expand_dims(smas, axis=1)]
print(test)
test = _scale(test)

[[ 1.20744  1.20748  1.20704  1.20741  1.20716]
 [ 1.20739  1.20755  1.20698  1.20716  1.2071 ]
 [ 1.20714  1.20744  1.20567  1.20569  1.20707]
 [ 1.20573  1.20673  1.20556  1.20575  1.20703]
 [ 1.20577  1.20529  1.20626  1.20515  1.20691]
 [ 1.20527  1.20577  1.20419  1.20457  1.20682]
 [ 1.20454  1.20541  1.20428  1.20501  1.20664]
 [ 1.20495  1.20552  1.20484  1.20529  1.20651]
 [ 1.20527  1.20541  1.20447  1.20474  1.2064 ]
 [ 1.20476  1.20831  1.20448  1.20521  1.20622]]


In [146]:
model.predict(np.expand_dims(test, axis=0))

array([[ 0.03107884]], dtype=float32)

In [147]:
values = test
np.c_[values, np.expand_dims(smas, axis=1)]

array([[ 1.        ,  0.45033113,  1.        ,  1.        ,  1.        ,
         1.20716   ],
       [ 0.96551724,  0.49668874,  0.95789474,  0.82394366,  0.87234043,
         1.2071    ],
       [ 0.79310345,  0.42384106,  0.03859649, -0.21126761,  0.80851064,
         1.20707   ],
       [-0.17931034, -0.04635762, -0.03859649, -0.16901408,  0.72340426,
         1.20703   ],
       [-0.15172414, -1.        ,  0.45263158, -0.5915493 ,  0.46808511,
         1.20691   ],
       [-0.49655172, -0.68211921, -1.        , -1.        ,  0.27659574,
         1.20682   ],
       [-1.        , -0.9205298 , -0.93684211, -0.69014085, -0.10638298,
         1.20664   ],
       [-0.71724138, -0.84768212, -0.54385965, -0.49295775, -0.38297872,
         1.20651   ],
       [-0.49655172, -0.9205298 , -0.80350877, -0.88028169, -0.61702128,
         1.2064    ],
       [-0.84827586,  1.        , -0.79649123, -0.54929577, -1.        ,
         1.20622   ]])